<a href="https://colab.research.google.com/github/Parsa2820/cs-project/blob/master/cs-project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [198]:
######################################
############### INPUTS ###############
######################################

# Task creation rate poisson distribution rate
X = 100
# Task execution time exponential distribution mean
Y = 10
# Maximum waiting time exponential distribution mean
Z = 100
# Simulation time
T = 1000
# Number of tasks
N = 10


In [199]:
######################################
############### CONFIG ###############
######################################

# Task priority distribution
P = {
    "3.low": 0.7,
    "2.normal": 0.2,
    "1.high": 0.1
}
# Limit of tasks in the second layer
K = 10
# First round robin time
T1 = 1
# Second round robin time
T2 = 1
# Second layer queues selcetion distribution
QUEUES_DISTRIBUTION = {
    "rr1": 0.8,
    "rr2": 0.1,
    "fcfs": 0.1
}


In [200]:
import numpy as np


In [201]:
class Task:
    def __init__(self):
        self.service_time = 0
        self.priority = 0
        self.arrival_time = 0
        self.max_waiting_time = 0

    def __repr__(self):
        return f"Task({self.service_time}, {self.priority}, {self.arrival_time})"


class CpuScheduler:
    def __init__(self, tasks):
        self.priority_queue = tasks
        self.round_robin_1_queue = []
        self.round_robin_2_queue = []
        self.fcfs_queue = []
        self.t = 0
        self.done_tasks = []
        self.timeout_tasks = []
        self.active_cpu_time = T

    def run(self):
        while self.t < T:
            self.remove_timeout_tasks()
            layer_2_tasks_count = len(self.round_robin_1_queue) + len(self.round_robin_2_queue) + len(self.fcfs_queue)
            if layer_2_tasks_count < K:
                self.job_loader(K - layer_2_tasks_count)
            layer_2_tasks_count = len(self.round_robin_1_queue) + len(self.round_robin_2_queue) + len(self.fcfs_queue)
            if layer_2_tasks_count == 0:
                self.t += 1
                self.active_cpu_time -= 1
            self.dispatcher()

    def job_loader(self, k):
        self.round_robin_1_queue += self.pop_tasks_from_priority_queue(k)

    def pop_tasks_from_priority_queue(self, k):
        queue = [x for x in self.priority_queue if x.arrival_time >= self.t]
        queue.sort(key=lambda x: x.priority)
        if len(queue) > k:
            self.priority_queue, result = queue[k:], queue[:k]
        else:
            self.priority_queue, result = [], queue
        result.sort(key=lambda x: x.arrival_time)
        return result

    def dispatcher(self):
        queue = np.random.choice(list(QUEUES_DISTRIBUTION.keys()), p=list(QUEUES_DISTRIBUTION.values()))
        if queue == "rr1":
            self.round_robin_1()
        elif queue == "rr2":
            self.round_robin_2()
        elif queue == "fcfs":
            self.fcfs()        

    def round_robin_1(self):
        if not self.round_robin_1_queue:
            return
        task = self.round_robin_1_queue.pop(0)
        if task.service_time > T1:
            task.service_time -= T1
            self.round_robin_2_queue.append(task)
        else:
            self.done_tasks.append(task)
        self.t += T1

    def round_robin_2(self):
        if not self.round_robin_2_queue:
            return
        task = self.round_robin_2_queue.pop(0)
        if task.service_time > T2:
            task.service_time -= T2
            self.fcfs_queue.append(task)
        else:
            self.done_tasks.append(task)
        self.t += T2

    def fcfs(self):
        if not self.fcfs_queue:
            return
        task = self.fcfs_queue.pop(0)
        self.done_tasks.append(task)
        self.t += task.service_time

    def remove_timeout_tasks(self):
        self.timeout_tasks += [x for x in self.priority_queue if x.arrival_time + x.max_waiting_time < self.t]
        self.timeout_tasks += [x for x in self.round_robin_1_queue if x.arrival_time + x.max_waiting_time < self.t]
        self.timeout_tasks += [x for x in self.round_robin_2_queue if x.arrival_time + x.max_waiting_time < self.t]
        self.timeout_tasks += [x for x in self.fcfs_queue if x.arrival_time + x.max_waiting_time < self.t]
        self.priority_queue = [x for x in self.priority_queue if x.arrival_time + x.max_waiting_time >= self.t]
        self.round_robin_1_queue = [x for x in self.round_robin_1_queue if x.arrival_time + x.max_waiting_time >= self.t]
        self.round_robin_2_queue = [x for x in self.round_robin_2_queue if x.arrival_time + x.max_waiting_time >= self.t]
        self.fcfs_queue = [x for x in self.fcfs_queue if x.arrival_time + x.max_waiting_time >= self.t]


In [202]:
def job_creator():
    tasks = []
    t = 0
    for i in range(N):
        task = Task()
        task.service_time = np.random.exponential(Y)
        task.priority = np.random.choice(list(P.keys()), p=list(P.values()))
        task.arrival_time = t
        t += np.random.poisson(X)
        task.max_waiting_time = np.random.exponential(Z)
        tasks.append(task)
    return tasks


In [203]:
tasks = job_creator()
cpu_scheduler = CpuScheduler(tasks)
cpu_scheduler.run()


In [204]:
######################################
############### OUTPUTS ##############
######################################

# Average queues length

# Average waiting times in queues

# CPU utilization
print(f"CPU utilization: {cpu_scheduler.active_cpu_time / T * 100}%")

# Percentage of timed out tasks

CPU utilization: 9.2%
